In [2]:
import pandas as pd
import numpy as np
import json
import os
pd.set_option('max_rows', 1000)

Littman, Justin; Kerchner, Daniel; Wrubel, Laura, 2017, "End of Term 2016 U.S. Government Twitter Archive", https://doi.org/10.7910/DVN/TQBLWZ, Harvard Dataverse, V1

In [4]:
government_agency = []
with open('twitter_accounts/eot-README.txt') as f:
    for line in f: 
        if '*' in line: 
            name = line.strip('*').strip()
            if '-' in name: name = name.split('-')[0]
            if ',' in name: name = name.split(',')[0]
            if ' ' in name: name = name.split(' ')[0]
            government_agency.append(name)
government_agency = set(government_agency)

Littman, Justin, 2017, "115th U.S. Congress Tweet Ids", https://doi.org/10.7910/DVN/UIVHQR, Harvard Dataverse, V5, UNF:6:pa6q3/72341rRixB7ez15Q== [fileUNF]

In [5]:
house115 = pd.read_csv('twitter_accounts/115thCongress/representatives-accounts-1.csv')
house115 = list(house115["Token"])
senate115 = pd.read_csv('twitter_accounts/115thCongress/senators-accounts-1.csv')
senate115 = list(senate115["Token"])

116th Congress, Github
[Read our blog for more details](https://ws-dl.blogspot.com/2019/04/2019-04-01-creating-data-set-for-116th.html)

In [6]:
Congress116 = pd.read_csv('twitter_accounts/116thCongress/116Congress.csv', 
                          encoding="ISO-8859-1")
sen116 = list(set(Congress116[Congress116['Position']=='Sen']['TweetCongress']))
rep116 = list(set(Congress116[Congress116['Position']=='Rep']['TweetCongress']))

Littman, Justin; Wrubel, Laura; Kerchner, Daniel; Bromberg Gaber, Yonah, 2017, "News Outlet Tweet Ids", https://doi.org/10.7910/DVN/2FIFLH, Harvard Dataverse, V3, UNF:6:I38WJ5vqwDky1fkEOeexvQ== [fileUNF]

In [7]:
news_outlet = pd.read_csv('twitter_accounts/new_outlets-accounts.csv')
news_outlet = list(set(news_outlet['Token']))

Littman, Justin; Kerchner, Daniel; Wrubel, Laura, 2017, "U.S. Government Tweet Ids", https://doi.org/10.7910/DVN/2N3HHD, Harvard Dataverse, V2, UNF:6:L8JDWfyfS9C694Nl7YfG7Q== [fileUNF]

In [8]:
gov_account = pd.read_csv('twitter_accounts/us_govt-accounts.csv')
gov_account = list(set(gov_account['Token']))
government_agency = list(set(list(government_agency) + list(gov_account)))

Wrubel, Laura; Littman, Justin; Kerchner, Dan, 2019, "2018 U.S. Congressional Election Tweet Ids", https://doi.org/10.7910/DVN/AEZPLU, Harvard Dataverse, V1, UNF:6:gxmogmaacqF8Mu3nvM793w== [fileUNF]

## Data Integration

In [ ]:
datasets = [house115, senate115, sen116, rep116, news_outlet, government_agency]
name = ["house115", "senate115", "sen116", "rep116", "news_outlet", "government_agency"]

account_types = dict()
for ind, n in enumerate(name):
    account_types[n] = datasets[ind]
    
json.dump(account_types, open('account_type.json', 'w'))

In [ ]:
def trans(x, tag):
    #if isinstance(x, float): return x
    if 'tweets' not in tag: cleaned_x = x.strip(f'{tag}\n{tag}\n').replace(',', '')
    else: cleaned_x = x.strip('Tweets\nTweets, current page.').replace(',', '')
    if cleaned_x.isnumeric(): return int(cleaned_x)
    else: 
        if 'K' not in cleaned_x: return cleaned_x
        cleaned_x = float(cleaned_x.strip('K'))
        cleaned_x = cleaned_x*1000
        return int(cleaned_x)

In [ ]:
def clean_user_info(original):
    #twitter_user_info = json.load(open('drafts/user_info_2.json')) 
    twitter_user_info = json.load(open(original))
    result = {}
    for d in twitter_user_info: 
        if d: result.update(d)
    
    updated_dict = dict()
    for d in result:
        is_in = ''
        if not d: continue
        if not result[d]: continue
        if 'Protected Tweets' in result[d][0]: continue
        if isinstance(result[d][4], float): continue
        
        for i, data in enumerate(datasets):
            if d in data: is_in = name[i]
    
        updated_dict[d] = dict()
        updated_dict[d]['fullname'] = result[d][0]
        updated_dict[d]['desc'] = result[d][1].strip()
        updated_dict[d]['location'] = result[d][2].strip()
        updated_dict[d]['join_time'] = result[d][3]
        updated_dict[d]['following'] = trans(result[d][4], 'Following')
        updated_dict[d]['follower'] = trans(result[d][5], 'Followers')
        updated_dict[d]['num_tweets'] = trans(result[d][6], 'tweets')
        updated_dict[d]['type'] = is_in
    
    return updated_dict

In [12]:
for d in result:
    is_in = ''
    if not d: continue
    if not result[d]: continue
    if 'Protected Tweets' in result[d][0]: continue
    if isinstance(result[d][4], float): continue
        
    for i, data in enumerate(datasets):
        if d in data: is_in = name[i]
    
    updated_dict[d] = dict()
    updated_dict[d]['fullname'] = result[d][0]
    updated_dict[d]['desc'] = result[d][1].strip()
    updated_dict[d]['location'] = result[d][2].strip()
    updated_dict[d]['join_time'] = result[d][3]
    updated_dict[d]['following'] = trans(result[d][4], 'Following')
    updated_dict[d]['follower'] = trans(result[d][5], 'Followers')
    updated_dict[d]['num_tweets'] = trans(result[d][6], 'tweets')
    updated_dict[d]['type'] = is_in

In [14]:
json.dump(updated_dict, open('cleaned_account_info.json', 'w')) 